In [ ]:
#import csv file
import pandas as pd
order_brushing_df = pd.read_csv("/order_brush_order.csv")

FileNotFoundError: ignored

In [ ]:
order_brushing_df.head()

NameError: ignored

In [ ]:
#knowing unique shopid
print(len(order_brushing_df['shopid'].unique()))

In [ ]:
order_groupby_shop = order_brushing_df.groupby('shopid') 

In [ ]:
#changing the event_time data type into datetime
from datetime import datetime
order_brushing_df['event_time'] = order_brushing_df['event_time'].apply(lambda x:datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [ ]:
order_brushing_df.head()

In [ ]:
# #filtering more than 1 time order
# df1 = order_brushing_df.groupby('shopid').filter(lambda x: len(x) > 1)

# #filter shopid based on orderid
# df1 = df1[~df1['shopid'].isin(shop_order_statistic['shopid'])]
df1 = order_brushing_df

In [ ]:
df1.head()

In [ ]:
#group by event time and shop id, and counting the amount of trasaction (based on order / user)
df1_shopid = df1.groupby([pd.Grouper(key='event_time',freq='H'),df1.shopid]).size().reset_index(name='count_order')
df1_userid = df1.groupby([pd.Grouper(key='event_time',freq='H'),df1.userid]).size().reset_index(name='count_user')

In [ ]:
#group by event_time and shopid, then ccount user that do the transaction in that time
df2 = df1.groupby([pd.Grouper(key='event_time',freq='H'),df1.shopid,df1.userid]).size().reset_index(name='count')

In [ ]:
df2_countuser=df2.groupby(['event_time','shopid'])['userid'].nunique().reset_index(name='count_user')

In [ ]:
#group by event_time and shop id, so can get the count_order and count_user that do the transaction
df_join = pd.merge(df1_shopid,df2_countuser,on=['event_time','shopid'],how='left')
df_join

In [ ]:
#getting value of concentrate_rate
df_join['concentrate_rate'] = df_join['count_order'] / df_join['count_user']
df_join

In [ ]:
# df_final = pd.merge(df_join,df2,on=['event_time','shopid'],how='left')

In [ ]:
#filter to get the potential of order brushing
df_brush_more_three = df_join[df_join['concentrate_rate'] >= 3]

In [ ]:
df_final = pd.merge(df_brush_more_three,df2,on=['event_time','shopid'],how='left')

In [ ]:
df_final = df_final[['shopid','userid']]
df_final = df_final.sort_values(by='shopid', ascending=False).reset_index()[['shopid', 'userid']]
#df_final = df_final.unique()
df_final['userid']= df_final['userid'].astype(str)

In [ ]:
df_final

In [ ]:
#configure the dataframe based on the requirement of the competition
df_final_fix = df_final.groupby(['shopid'])['userid'].apply(lambda x: '&'.join(x.unique())).reset_index()

In [ ]:
df_final_fix[df_final_fix['userid'].str.contains("&")]

In [ ]:
df_brush_less_three = df_join[(df_join['concentrate_rate'] < 3)]
df_brush_less_three = df_brush_less_three[~df_brush_less_three.shopid.isin(df_final_fix.shopid)]            

In [ ]:
#filter only id with concentrate rate less than 3 
shopidless = df_brush_less_three['shopid'].unique()
zero = [str(0) for i in range(len(shopidless))]
df_con_less_three = pd.DataFrame(list(zip(shopidless, zero)), columns =['shopid', 'userid']) 

In [ ]:
df_final_done = pd.concat([df_final_fix,df_con_less_three])

In [ ]:
len(df_final_fix) + len(df_con_less_three) == len(order_brushing_df['shopid'].unique())

In [ ]:
df_final_done = df_final_done[df_final_done['userid'].str.contains("&")].reset_index()
df_final_done = df_final_done[['shopid','userid']]

In [ ]:
df_final_done

In [ ]:
df_final_done.to_csv(r'/order_brushing_sixthfloor.csv')